In [154]:
import pandas as pd, seaborn as sns
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import time
import smtplib

%matplotlib inline

### Quick Install:  Selenium + Webdriver

The official install guide is here:
https://pypi.python.org/pypi/selenium


Also, you will need to download a file and load it with python for *Chome Webdriver*.  Basically, for your given platform (Mac, Ubuntu, or Windows), there is a file you need to download then reference from selenium in Python in order to get it to work.  Choose the file for your given platform here:

https://chromedriver.storage.googleapis.com/index.html?path=2.35/

>  You may have to chmod 770 or 777 the webdriver executable

In [155]:
path_to_chromedriver = './../chromedriver' # change path as needed
driver = webdriver.Chrome(executable_path = path_to_chromedriver)

In [157]:
data = pd.read_csv('./data/coinTelegraph_feb13.csv')
data.rename(columns= {'Unnamed: 0':'article_id'}, inplace =True)

In [159]:
articles = []
tags = []
links = []

for _, d in data.iterrows():
    
    article_id = d['article_id']
    url = d['url']
    
    result  = driver.get(url)
    html = driver.page_source;
    soup = BeautifulSoup(html, "lxml")

    
    article = {}

    # - main stuff
    article['article_id'] = article_id
    article['text'] = soup.find('div', {'class': 'post-full-text'}).text.strip()
    article['views'] = soup.find('div', {'class': 'total-views'}).find('span', {'class': 'total-qty'}).text.strip()
    article['shares'] = soup.find('div', {'class': 'total-shares'}).find('span', {'class': 'total-qty'}).text.strip()

    # - tags 
    tg = soup.find('div',{'class':'tags'})

    for tag in tg.findAll('li'):
        tags.append((article_id, tag.text.strip()))

    # - links 
    body = soup.find('div', {'class': 'post-full-text'})

    for a in body.findAll('a'):
        print(a.attrs['href'])
        links.append((article_id, a.attrs['href']))

    articles.append(article)


    # - snap article to to main data frame
  
    break

https://cointelegraph.com/news/binance-exchange-halts-trading-withdrawals-over-server-issue-assures-no-hack
https://twitter.com/binance_2017/status/961635473591435265
https://cointelegraph.com/news/530-million-in-xem-stolen-from-coincheck-can-be-traced-nem-team-confirms
https://cointelegraph.com/news/bitgrail-exchange-asks-devs-of-stolen-coin-to-alter-ledger-to-cover-losses
https://cointelegraph.com/news/japan-coincheck-exchange-freezes-all-withdrawals-as-up-to-723-mln-leaves-its-wallet
https://www.linkedin.com/pulse/binance-incident-recap-changpeng-zhao/


In [160]:
tags_df     =  pd.DataFrame(tags, columns = ['article_id','tag'])
links_df    =  pd.DataFrame(links, columns = ['article_id','link'])
articles_df =  pd.DataFrame(articles)

In [165]:
folder = 'data'

tags_df.to_csv(folder+'/tags.csv')
links_df.to_csv(folder+'/links.csv')
articles_df.to_csv(folder+'/articles.csv')